In [1]:
import os, sys
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
def image_paste(image1, image2):
    '''paste image2 (PIL) onto image1 (PIL) (images have the same shape)'''
    image1.paste(image2, image2)
    return image1
def images_paste(images_list):
    '''paste list of images (PIL)'''
    num_images = len(images_list)
    assert num_images != 0
    if num_images == 1:
        return images_list[0]
    else:
        for i in range(num_images,1,-1):
            images_list[i-2].paste(images_list[i-1], images_list[i-1])
        return images_list[0]

def bin_mask_paste(mask1, mask2):
    '''combine 2 binary masks (PIL mode 'L') together'''
    mask1_np, mask2_np = np.array(mask1), np.array(mask2)
    mask_np = np.logical_or(mask1_np, mask2_np)
    return Image.fromarray(mask_np.astype(np.uint8))
def bin_masks_paste(masks_list):
    '''combine a list of binary masks (PIL mode 'L')'''
    num_masks = len(masks_list)
    assert num_masks != 0
    if num_masks == 1:
        return masks_list[0]
    else:
        mask = np.zeros(np.array(masks_list[0]).shape, dtype=np.uint8)
        for i in range(num_masks):
            mask = bin_mask_paste(mask, masks_list[i])
        return mask

In [3]:
content_list = os.listdir('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18')
tool_list = []
for i in content_list:
    if os.path.isdir('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'+i):
        tool_list.append(i)

aug_images_dict = {}
# sys.exit(0)
for tool in tool_list:
    file_path = os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18', tool)
    image_name_list = os.listdir(os.path.join(file_path, 'image_2_50'))
    aug_images_dict[tool] = image_name_list
tool_list
# aug_images_dict

['3-Large_Needle_Driver',
 '5-Ultrasound_Probe',
 '6-Suction_Instrument',
 '1-Bipolar_Forceps',
 '4-Monopolar_Curved_Scissors',
 '7-Clip_Applier',
 '2-Prograsp_Forceps']

In [7]:
# multiple instruments
blended_root = '/mnt/data-ssd/wa/overlay/FgBgAug/blended_2base50_multi_instru_bgold/'
fg_root_path = '/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'
# bg_name_list = os.listdir('/mnt/data-ssd/wa/overlay/FgBgAug/bg/aug')
num = 0
seed = 0
for tool in tool_list:
    file_path = os.path.join(fg_root_path, tool)
    image_name_list = os.listdir(os.path.join(file_path, 'image_2_50'))
    # sample 20 from 50 (40%)
    np.random.seed(seed)
    seed+=1
    np.random.shuffle(image_name_list)
    image_name_list = image_name_list[:int(len(image_name_list)*0.4)]
    for i in tqdm(range(len(image_name_list)), ncols=100, desc=tool[0]):
        fg_i = Image.open(os.path.join(file_path, 'image_2_50', image_name_list[i])).convert('RGBA')
        mask_i = Image.open(os.path.join(file_path, 'mask_2_50', image_name_list[i]))
        fg_list, mask_list = [], []
        file_prefix = tool[0]
        fg_list.append(fg_i)
        mask_list.append(mask_i)
        np.random.seed(seed)
        seed+=1
        rand_p = np.random.uniform()
        # rand_p = 0.9
        if rand_p < 0.8: # 1 instru
            num_instru = 1
        # elif rand_p > 0.7: # 3 instru
        #     num_instru = 3
        else: # 2 instru
            num_instru = 2
        if num_instru > 1:
            cand_tool_list = tool_list[:]
            cand_tool_list.remove(tool)
            # print(cand_tool_list)
            cand_tools = np.random.choice(cand_tool_list,num_instru-1,replace=False)
            for i in range(num_instru-1):
                cand_image_name = np.random.choice(aug_images_dict[cand_tools[i]])
                cand_image = Image.open(os.path.join(fg_root_path, cand_tools[i], 'image_2_50', cand_image_name)).convert('RGBA')
                cand_mask = Image.open(os.path.join(fg_root_path, cand_tools[i], 'mask_2_50', cand_image_name))
                fg_list.append(cand_image)
                mask_list.append(cand_mask)
                file_prefix+=cand_tools[i][0]
        assert len(fg_list) == num_instru
        assert len(mask_list) == num_instru
        fg, mask = images_paste(fg_list), bin_masks_paste(mask_list) 
        # sample 20 from 50 (40%)
        np.random.seed(seed)
        seed+=1
        bg_name_list = os.listdir('/mnt/data-ssd/wa/overlay/FgBgAug/bg/aug')
        np.random.shuffle(bg_name_list)
        bg_name_list = bg_name_list[:20]
        for bg_name in bg_name_list:
            bg = Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/bg/aug', bg_name)).convert('RGBA')
            bg.paste(fg,fg)
            # save mask and blended
            # convert to binary mask
            mask_np = np.array(mask)
            mask_np[mask_np!=0]=255
            mask = Image.fromarray(mask_np)

            mask.save(os.path.join(blended_root, 'annotations', 'binary', file_prefix+'-'+image_name_list[i][:-4]+'-'+bg_name))
            bg.save(os.path.join(blended_root, 'images', file_prefix+'-'+image_name_list[i][:-4]+'-'+bg_name))
            num+=1
            # sys.exit(0)
print('Total number:',num)

2: 100%|████████████████████████████████████████████████████████████| 40/40 [07:17<00:00, 10.94s/it]

Total number: 6400
